In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ibm-hr-analytics-attrition-dataset/WA_Fn-UseC_-HR-Employee-Attrition.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df=pd.read_csv('/kaggle/input/ibm-hr-analytics-attrition-dataset/WA_Fn-UseC_-HR-Employee-Attrition.csv')
df.head(5)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [4]:
df.describe(include=['bool','object'])

,Attrition,BusinessTravel,Department,EducationField,Gender,JobRole,MaritalStatus,Over18,OverTime
count,1470,1470,1470,1470,1470,1470,1470,1470,1470
unique,2,3,3,6,2,9,3,1,2
top,No,Travel_Rarely,Research & Development,Life Sciences,Male,Sales Executive,Married,Y,No
freq,1233,1043,961,606,882,326,673,1470,1054


In [5]:
df['Attrition']=df.Attrition.astype('category').cat.codes

In [6]:
df.drop(['EmployeeCount','EmployeeNumber','Over18', 'StandardHours'], axis=1,inplace=True)

In [7]:
categorical_columns=[]

for col in df.columns:
    if df[col].dtypes == object and len(df[col].unique())<=50:
        categorical_columns.append(col)

categorical_columns

['BusinessTravel',
 'Department',
 'EducationField',
 'Gender',
 'JobRole',
 'MaritalStatus',
 'OverTime']

In [8]:
from sklearn.preprocessing import LabelEncoder

label=LabelEncoder()

for col in categorical_columns:
    df[col]=label.fit_transform(df[col])

In [9]:
from sklearn.model_selection import train_test_split

X=df.drop('Attrition', axis=1)
y=df['Attrition']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [10]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

def print_score(clf, X_train, y_train, X_test, y_test, train=True):
    if train:
        pred = clf.predict(X_train)
        clf_report = pd.DataFrame(classification_report(y_train, pred, output_dict=True))
        print("Train Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(y_train, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_train, pred)}\n")
        
    elif train==False:
        pred = clf.predict(X_test)
        clf_report = pd.DataFrame(classification_report(y_test, pred, output_dict=True))
        print("Test Result:\n================================================")        
        print(f"Accuracy Score: {accuracy_score(y_test, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_test, pred)}\n")

In [11]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(n_estimators=20)
gb.fit(X_train, y_train)


GradientBoostingClassifier(n_estimators=20)

In [12]:
print_score(gb, X_train, y_train, X_test, y_test, train=True)
print_score(gb, X_train, y_train, X_test, y_test, train=False)

Train Result:
Accuracy Score: 88.24%
_______________________________________________
CLASSIFICATION REPORT:
                    0           1  accuracy    macro avg  weighted avg
precision    0.878882    0.936508   0.88241     0.907695      0.888738
recall       0.995311    0.335227   0.88241     0.665269      0.882410
f1-score     0.933480    0.493724   0.88241     0.713602      0.858264
support    853.000000  176.000000   0.88241  1029.000000   1029.000000
_______________________________________________
Confusion Matrix: 
 [[849   4]
 [117  59]]

Test Result:
Accuracy Score: 86.62%
_______________________________________________
CLASSIFICATION REPORT:
                    0          1  accuracy   macro avg  weighted avg
precision    0.872390   0.600000  0.866213    0.736195      0.834712
recall       0.989474   0.098361  0.866213    0.543917      0.866213
f1-score     0.927250   0.169014  0.866213    0.548132      0.822370
support    380.000000  61.000000  0.866213  441.000000    441.

In [13]:
imp_features=pd.DataFrame({
    "Varname":X_train.columns,
    "Imp":gb.feature_importances_
})
imp_features

,Varname,Imp
0,Age,0.096971
1,BusinessTravel,0.000000
2,DailyRate,0.025710
3,Department,0.000000
4,DistanceFromHome,0.022910
5,Education,0.000000
6,EducationField,0.011558
7,EnvironmentSatisfaction,0.027229
8,Gender,0.000000
9,HourlyRate,0.007817


In [16]:
n_estimators=[int(x) for x in np.linspace(start=100, stop=1000, num=10)]
max_features=['sqrt', 'log2']
max_depth= [int(x) for x in np.linspace(start=10,stop=110, num=11)]
max_depth.append(None)
min_samples_split=[2,5,10]
min_samples_leaf=[1,2,4]
learning_rate=[0.1,0.05, 0.01]

In [21]:
from sklearn.model_selection import RandomizedSearchCV
random_grid={
    'n_estimators': n_estimators,
    'max_features': max_features,
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
    'learning_rate' : learning_rate
}

gb_clf= GradientBoostingClassifier(random_state=42)

gb_cv = RandomizedSearchCV(
    estimator=gb_clf,
    scoring = 'f1',
    param_distributions=random_grid,
    n_iter=200,
    cv=5,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

gb_cv.fit(X_train,y_train)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


RandomizedSearchCV(cv=5, estimator=GradientBoostingClassifier(random_state=42),
                   n_iter=200, n_jobs=-1,
                   param_distributions={'learning_rate': [0.1, 0.05, 0.01],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['sqrt', 'log2'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500, 600, 700, 800,
                                                         900, 1000]},
                   random_state=42, scoring='f1', verbose=1)

In [22]:
best_params= gb_cv.best_params_
best_params

{'n_estimators': 200,
 'min_samples_split': 10,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 110,
 'learning_rate': 0.1}

In [23]:
gb_clf=GradientBoostingClassifier(**best_params)
gb_clf.fit(X_train,y_train)



GradientBoostingClassifier(max_depth=110, max_features='sqrt',
                           min_samples_leaf=4, min_samples_split=10,
                           n_estimators=200)

In [25]:
print_score(gb_clf, X_train, y_train, X_test, y_test, train=True)
print_score(gb_clf, X_train, y_train, X_test, y_test, train=False)

Train Result:
Accuracy Score: 100.00%
_______________________________________________
CLASSIFICATION REPORT:
               0      1  accuracy  macro avg  weighted avg
precision    1.0    1.0       1.0        1.0           1.0
recall       1.0    1.0       1.0        1.0           1.0
f1-score     1.0    1.0       1.0        1.0           1.0
support    853.0  176.0       1.0     1029.0        1029.0
_______________________________________________
Confusion Matrix: 
 [[853   0]
 [  0 176]]

Test Result:
Accuracy Score: 87.53%
_______________________________________________
CLASSIFICATION REPORT:
                    0          1  accuracy   macro avg  weighted avg
precision    0.882353   0.687500  0.875283    0.784926      0.855400
recall       0.986842   0.180328  0.875283    0.583585      0.875283
f1-score     0.931677   0.285714  0.875283    0.608696      0.842326
support    380.000000  61.000000  0.875283  441.000000    441.000000
_______________________________________________
Conf